In [0]:
from google.colab import auth, drive
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.discovery import build
auth.authenticate_user()
drive.mount('/content/gdrive')

def path(fname):
    return 'gdrive/My Drive/Colab Notebooks/Avito/' + fname

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
train_data = pd.read_csv(path('new_train.csv'), index_col=0, na_filter=False)
train_data.head()

,title,description,price,category_id
item_id,,,,
0,картин,гобел размер 139х84см,1000.0,19
1,стул прессова кож,прод недор стул светл прессова кож стильн ножк...,1250.0,22
2,домашн мин бан,мин бан мб мин саун предназнач принят теплов п...,13000.0,37
3,эксклюзивн коллекц книг трансаэр подарок,прод эксклюзивн коллекц книг выпущен ограничен...,4000.0,43
4,ноутбук aser,прода ноутбук acer e5 511c2ta купл конц ноябр ...,19000.0,1


In [0]:
test_data = pd.read_csv(path('new_test.csv'), index_col=0, na_filter=False)
test_data.head()

,title,description,price
item_id,,,
489517,стоик журнальн стал,прод журнальн столик изготавлива стол полирова...,10000.0
489518,iphon 64gb,телефон хорош состоян комплект гарант салон со...,12500.0
489519,утеплител,теплопел лидер тепл толщин утеплител 20 мм 30 ...,250.0
489520,пальт демисезон,прод пальт женск букл отличн состоян длин изде...,1700.0
489521,samsung syncmaster t200n,условн рабоч проблем панел настройк монитор пе...,1000.0


In [0]:
categories = pd.read_csv(path('new_categories.csv'))
categories.head()

,category_id,name,subname,subsubname,subsubsubname
0,0,Бытовая электроника,Телефоны,iPhone,0
1,1,Бытовая электроника,Ноутбуки,0,0
2,2,Бытовая электроника,Телефоны,Samsung,0
3,3,Бытовая электроника,Планшеты и электронные книги,Планшеты,0
4,4,Бытовая электроника,"Игры, приставки и программы",Игровые приставки,0


In [0]:
!rm -rf fastText
!apt-get install cmake
!git clone https://github.com/facebookresearch/fastText.git; cd fastText; mkdir build && cd build && cmake ..;make && make install
!pip install fasttext

In [0]:
train_fasttext = pd.Series(train_data['title'] + ' ' + train_data['description'] + '\t' + '__label__' + train_data['category_id'].astype(str) + '\n')
test_fasttext = pd.Series(test_data['title'] + ' ' + test_data['description']  + '\n')

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train_fasttext, range(train_fasttext.size), test_size=0.33)

In [0]:
X_train.to_csv('X_train_fasttext.csv', header=False, index=False)
X_test.to_csv('X_test_fasttext.csv', header=False, index=False)

In [0]:
!fasttext supervised -input X_train_fasttext.csv -output fasttext_model -minn 3 -maxn 10 -wordNgrams 10 -dim 700 -epoch 12 -thread 2 -verbose 2 -lr 1.2

Read 12M words
Number of words:  295405
Number of labels: 54
tcmalloc: large alloc 6427140096 bytes == 0x563372e84000 @  0x7f6b64401887 0x5633648c4dfd 0x5633648d87c8 0x5633648dfb74 0x5633648bc882 0x5633648ba7f7 0x7f6b6349eb97 0x5633648bacda
Progress: 100.0% words/sec/thread:    7794 lr:  0.000000 loss:  0.184507 ETA:   0h 0m


In [0]:
!fasttext quantize -input X_train_fasttext.txt -output fasttext_model

tcmalloc: large alloc 6427140096 bytes == 0x55c51d186000 @  0x7ff16e431887 0x55c5169108bf 0x55c516924b84 0x55c516925671 0x55c5169085e3 0x55c516906a4d 0x7ff16d4ceb97 0x55c516906cda


In [0]:
!fasttext test fasttext_model.ftz X_test_fasttext.csv 1

N	161541
P@1	0.876
R@1	0.876


In [0]:
!fasttext supervised -input X_train_fasttext.csv -output fasttext_model2 -minn 3 -maxn 10 -wordNgrams 8 -dim 800 -epoch 10 -thread 2 -verbose 2 -lr 0.6

Read 12M words
Number of words:  296015
Number of labels: 54
tcmalloc: large alloc 7347249152 bytes == 0x5605606d4000 @  0x7f2bbb6eb887 0x56055106bdfd 0x56055107f7c8 0x560551086b74 0x560551063882 0x5605510617f7 0x7f2bba788b97 0x560551061cda
Progress: 100.0% words/sec/thread:    7502 lr:  0.000000 loss:  0.288004 ETA:   0h 0m


In [0]:
!fasttext quantize -input X_train_fasttext.txt -output fasttext_model2

tcmalloc: large alloc 7347249152 bytes == 0x563b2e5b4000 @  0x7ff513dbb887 0x563b27bad8bf 0x563b27bc1b84 0x563b27bc2671 0x563b27ba55e3 0x563b27ba3a4d 0x7ff512e58b97 0x563b27ba3cda


In [0]:
!fasttext test fasttext_model2.ftz X_test_fasttext.csv 1

N	161541
P@1	0.874
R@1	0.874


In [0]:
!ls -l --block-size=M

total 9027M
-rw-r--r--  1 root root    1M May  6 08:39 adc.json
drwxr-xr-x 13 root root    1M May  6 08:39 fastText
-rw-r--r--  1 root root 6136M May  6 11:24 fasttext_model.bin
-rw-r--r--  1 root root  773M May  6 11:49 fasttext_model.ftz
-rw-r--r--  1 root root 1935M May  6 11:26 fasttext_model.vec
drwx------  3 root root    1M May  6 08:39 gdrive
drwxr-xr-x  1 root root    1M Apr 29 16:32 sample_data
-rw-r--r--  1 root root   61M May  6 08:41 X_test_fasttext.csv
-rw-r--r--  1 root root  123M May  6 08:41 X_train_fasttext.csv
